## Running Chains on Traced Datasets

Developing applications with language models can be uniquely challenging. To manage this complexity and ensure reliable performance, LangChain provides tracing and evaluation functionality through . This notebook demonstrates how to run Chains, which are language model functions, on previously captured datasets or traces. Some common use cases for this approach include:

- Running an evaluation chain to grade previous runs.
- Comparing different chains, LLMs, and agents on traced datasets.
- Executing a stochastic chain multiple times over a dataset to generate metrics before deployment.

Please note that this notebook assumes you have LangChain+ tracing running in the background. To set it up, follow the [tracing directions here](..\/..\/tracing\/local_installation.md).


In [1]:
from langchain.client import LangChainClient

client = LangChainClient()

## Seed an example dataset

If you have been using LangChainPlus already, you may have datasets available. You can generate these from `Run`'s captured through the LangChain tracing API.

```
datasets = client.list_datasets()
datasets
```

Assuming you're running locally for the first time, you may not have runs stored, so we'll first upload an example evaluation dataset.

In [2]:
# !pip install datasets > /dev/null
# !pip install pandas > /dev/null

In [ ]:
import pandas as pd
from langchain.evaluation.loading import load_dataset

dataset = load_dataset("agent-search-calculator")
df = pd.DataFrame(dataset, columns=["question", "answer"])
df.columns = ["input", "output"] # The chain we want to evaluate below expects inputs with the "input" key 

In [4]:
df.head()

,input,output
0,How many people live in canada as of 2023?,"approximately 38,625,801"
1,who is dua lipa's boyfriend? what is his age r...,her boyfriend is Romain Gravas. his age raised...
2,what is dua lipa's boyfriend age raised to the...,her boyfriend is Romain Gravas. his age raised...
3,how far is it from paris to boston in miles,"approximately 3,435 mi"
4,what was the total number of points scored in ...,approximately 2.682651500990882


In [5]:
dataset_name = f"calculator_example.csv"

In [6]:
if dataset_name not in set([dataset.name for dataset in client.list_datasets()]):
    dataset = client.upload_dataframe(df, 
                            name=dataset_name,
                            description="Acalculator example dataset",
                            input_keys=["input"],
                            output_keys=["output"],
                   )

## Running a Chain on a Traced Dataset

Once you have a dataset, you can run a chain over it to see its results. The run traces will automatically be associated with the dataset for easy attribution and analysis.

**First, we'll define the chain we wish to run over the dataset.**

In this case, we're using an agent, but it can be any simple chain.

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0)
tools = load_tools(['serpapi', 'llm-math'], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False)

**Now we're ready to run the chain!**

In [8]:
chain_results = await client.arun_chain_on_dataset(
    dataset_name=dataset_name,
    chain=agent,
    batch_size=5, # Optional, sets the number of examples to run at a time
    session_name="Calculator Dataset Runs", # Optional. Will be sent to the 'default' session otherwise
)

Chain failed for example e290c4f5-0d56-4848-91e6-d794cfeed276. Error: 'age'. Please try again with a valid numerical expression
Chain failed for example ec590bea-8b5b-429b-a9f4-7d035e66853e. Error: unknown format from LLM: Assuming we don't have any information about the actual number of points scored in the 2023 super bowl, we cannot provide a mathematical expression to solve this problem.
Chain failed for example a933211c-83c0-43c4-b22f-40e088263849. Error: invalid syntax. Perhaps you forgot a comma? (<expr>, line 1). Please try again with a valid numerical expression
Chain failed for example b925eaa0-6b63-45c1-a3c3-c11ad53ca0eb. Error: 'VariableNode' object is not callable. Please try again with a valid numerical expression


In [9]:
chain_results

['The current population of Canada as of May 3, 2023 is 38,677,281.',
 "Anwar Hadid's age raised to the 0.43 power is approximately 3.87.",
 {'Error': "'age'. Please try again with a valid numerical expression"},
 'The distance between Paris and Boston is approximately 3448 miles.',
 {'Error': "unknown format from LLM: Assuming we don't have any information about the actual number of points scored in the 2023 super bowl, we cannot provide a mathematical expression to solve this problem."},
 {'Error': 'invalid syntax. Perhaps you forgot a comma? (<expr>, line 1). Please try again with a valid numerical expression'},
 '0',
 '1.9347796717823205',
 "Devin Booker is Kendall Jenner's boyfriend and his height (in inches) raised to .13 power is approximately 1.7589.",
 {'Error': "'VariableNode' object is not callable. Please try again with a valid numerical expression"}]

In [10]:
# You can navigate to the UI by clicking on the link below
client

LangChainClient (API URL: http://localhost:8000)